Sequential Chats and Customer Onboarding

## Setup

In [1]:
import os
api_key = os.environ["MISTRAL_API_KEY"]
llm_config = { "model": "mistral-large-latest", "api_key": api_key, "api_type": "mistral"}

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:
from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)


## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [4]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [5]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\Pandiyan\anaconda3\envs\agentai\Lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Pandi from Sivakasi


customer_proxy_agent (to onboarding_personal_information_agent):

Pandi from Sivakasi

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (f9f119f3-de61-4c08-9576-3875ea253971): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{"name":"Pandi","location":"Sivakasi"}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Gospels from Bible


customer_proxy_agent (to onboarding_topic_preference_agent):

Gospels from Bible

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (12f2ebc1-9999-4484-b597-fb5f84d5b1f9): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
{"name":"Pandi","location":"Sivakasi"}
Interested in reading about Gospels from the Bible.

--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Hello Pandi from Sivakasi! It's great to meet you. I'm excited to help you find some fun and engaging content about the Gospels from the Bible. Did you know that the word "Gospel" comes from the Old English word "godspell," which me

## Print out the summary

In [6]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

{'content': '{"name":"Pandi","location":"Sivakasi"}', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'Interested in reading about Gospels from the Bible.', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': "User is interested in reading about the Gospels from the Bible.\nProvided a fun fact about the symbols of the Gospels.\nShared a light-hearted joke.\nHighlighted the story of the feeding of the 5,000.\nOffered to provide more information based on user's preferences.", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}




In [10]:
summary[0]

{'content': '{"name":"Pandi","location":"Sivakasi"}',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [ ]:
#!pip install pandas

In [12]:
import pandas as pd
import ast
import json
import re

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0].get("content")
print(data_str)

data_dict = ast.literal_eval(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame([data_dict])

df


{"name":"Pandi","location":"Sivakasi"}


,name,location
0,Pandi,Sivakasi


In [13]:
summary[1]

{'content': 'Interested in reading about Gospels from the Bible.',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [14]:
df["TopicPreference"]=summary[1].get("content")

In [15]:
df

,name,location,TopicPreference
0,Pandi,Sivakasi,Interested in reading about Gospels from the B...


In [16]:
summary[2]

{'content': "User is interested in reading about the Gospels from the Bible.\nProvided a fun fact about the symbols of the Gospels.\nShared a light-hearted joke.\nHighlighted the story of the feeding of the 5,000.\nOffered to provide more information based on user's preferences.",
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [29]:
chat_results[2].chat_history[1].get("content")

'Hello Pandi from Sivakasi! It\'s great to meet you. I\'m excited to help you find some fun and engaging content about the Gospels from the Bible. Did you know that the word "Gospel" comes from the Old English word "godspell," which means "good news"?\n\nLet\'s start with a fascinating fact: The Gospels of Matthew, Mark, Luke, and John are each represented by a unique symbol in Christian art and literature. Matthew is often depicted as a winged man or angel, Mark as a winged lion, Luke as a winged ox or bull, and John as an eagle. These symbols are derived from the prophet Ezekiel\'s vision in the Old Testament.\n\nNow, here\'s a light-hearted joke for you:\n\nWhy don’t they play poker in the jungle?\n\nBecause there are too many cheetahs!\n\nNow, let\'s dive into an interesting story from the Gospels. One of the most famous stories is the feeding of the 5,000, which appears in all four Gospels. Imagine being part of that crowd, witnessing Jesus multiply a small amount of food to feed 

In [30]:
df["Enagement"]=chat_results[2].chat_history[1].get("content")

In [31]:
df

,name,location,TopicPreference,Enagement
0,Pandi,Sivakasi,Interested in reading about Gospels from the B...,Hello Pandi from Sivakasi! It's great to meet ...


In [24]:
ll=df.tail(1).values

In [25]:
ll

array([['Pandi', 'Sivakasi',
        'Interested in reading about Gospels from the Bible.',
        'Let\'s find something fun to read.\nContext: \n{"name":"Pandi","location":"Sivakasi"}\nInterested in reading about Gospels from the Bible.']],
      dtype=object)

In [ ]:
#!pip install gspread
#!pip install oauth2client

In [26]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials

    # Define the scope for Google Sheets API
    scope = ['https://www.googleapis.com/auth/spreadsheets']

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'fair-subset-455717-s6-84dee767824c.json', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '1gLkaCmWgQp_fuEqUeOkf-SsjwYfh4PuFQ2tpEKoLTFI'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist())

    print("Data added successfully!")
    return "added"


In [32]:
add_leads_to_google_sheet(df)

Data added successfully!


'added'

## Print out the cost

In [28]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 2.0999999999999995e-05, 'mistral-large-latest': {'cost': 2.0999999999999995e-05, 'prompt_tokens': 55, 'completion_tokens': 15, 'total_tokens': 70}}, 'usage_excluding_cached_inference': {'total_cost': 2.0999999999999995e-05, 'mistral-large-latest': {'cost': 2.0999999999999995e-05, 'prompt_tokens': 55, 'completion_tokens': 15, 'total_tokens': 70}}}


{'usage_including_cached_inference': {'total_cost': 2.31e-05, 'mistral-large-latest': {'cost': 2.31e-05, 'prompt_tokens': 64, 'completion_tokens': 13, 'total_tokens': 77}}, 'usage_excluding_cached_inference': {'total_cost': 2.31e-05, 'mistral-large-latest': {'cost': 2.31e-05, 'prompt_tokens': 64, 'completion_tokens': 13, 'total_tokens': 77}}}


{'usage_including_cached_inference': {'total_cost': 0.00027929999999999996, 'mistral-large-latest': {'cost': 0.00027929999999999996, 'prompt_tokens': 521, 'completion_tokens': 410, 'total_tokens': 931}}, 'usage_excluding_cached_inference': {'total_co